# Run REopt API located on localhost or NREL server

## Initialization

In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import copy
import os
from results_poller import reo_optimize

## Load a previously saved API response .json file instead of running REopt

In [2]:
response_json = 'response_1'
with open('saved_responses/' + response_json + '.json', 'rb') as handle:
    api_response = json.load(handle)

## Scenario Inputs (Post), if wanting to do a new API call

In [12]:
post_1 = {
  "Scenario": {
    "timeout_seconds": 400,
    "optimality_tolerance_techs": 0.001,
    "time_steps_per_hour": 1,
    "Site": {
      "latitude": 37.78,
      "longitude": -122.45,
      "address": "",
      "land_acres": None,
      "roof_squarefeet": None,
      "elevation_ft": 700.0,
      "Financial": {
        "om_cost_escalation_pct": 0.025,
        "escalation_pct": 0.023,
        "boiler_fuel_escalation_pct": 0.034,
        "offtaker_tax_pct": 0.26,
        "offtaker_discount_pct": 0.083,
        "third_party_ownership": False,
        "owner_tax_pct": 0.26,
        "owner_discount_pct": 0.083,
        "analysis_years": 25,
        "value_of_lost_load_us_dollars_per_kwh": 100.0,
        "microgrid_upgrade_cost_pct": 0.3
      },
      "LoadProfile": {
        "doe_reference_name": "Hospital",
      },
      "LoadProfileBoilerFuel": {
        "doe_reference_name": "Hospital"
      },
      "LoadProfileChillerThermal": {
        "doe_reference_name": "Hospital"
      },      
      "ElectricTariff": {
        "net_metering_limit_kw": 0.0,
        "interconnection_limit_kw": 100000000.0,
        "urdb_label": "5e1676e95457a3f87673e3b0"
      },
      "FuelTariff": {
        "existing_boiler_fuel_type": "natural_gas",
        "boiler_fuel_blended_annual_rates_us_dollars_per_mmbtu": 11.0
      },
      "GHP": {
        "building_sqft": 50000.0,
        "force_ghp": True
      },
      "PV": {
        "max_kw": 0.0
      },
      "Storage": {
        "max_kw": 0.0,
        "max_kwh": 0.0
      }
    }
  }
}

## Save the post to a .json file for sharing or future loading in

In [13]:
# Convert python dictionary post into json and save to a .json file
post_save = post_1
with open("saved_posts/" + 'post_1.json', 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the post

In [4]:
# Load a json into a python dictionary
load_post = "post_1"
with open("saved_posts/" + load_post + ".json", 'r') as fp:
    post_1 = json.load(fp)

### Enter the root_url based on the "location" of the server that's hosting the API
### and your API key from developer.nrel.gov

In [15]:
root_url = "http://localhost:8000"
API_KEY = "uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo"

## POST and poll (period GET request) API to get a new result, if not loading in a previous response
## ...This may take a while!

In [16]:
api_response = reo_optimize(post_1, API_KEY, root_url)

Response OK from http://localhost:8000/v1/job/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo.
Polling http://localhost:8000/v1/job/6d961321-a86a-42fa-99a1-07fc8d504306/results/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo for results with interval of 10s...


### If you get disconnected from the polling function but you think it ran, 
### copy the run_uuid from the log above to manually GET the results:

In [10]:
run_uuid = "10816148-36c0-4694-a96b-906416c0cc22"
results_url = root_url + '/v1/job/' + run_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
api_response = json.loads(resp.text)

## Get summary of results

In [24]:
print("NPV ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["npv_us_dollars"])
print("Capital Cost, Net ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs"])
tech_list = ["PV", "Wind", "Storage", "CHP", "Generator", "HotTES", "ColdTES", "AbsorptionChiller", "GHP", "NewBoiler", "SteamTurbine"]
for tech in tech_list:
    if tech in post_1["Scenario"]["Site"].keys():
        if tech == "GHP":
            print("GHX Number of Boreholes = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("number_of_boreholes"))
            print("GHP Heat Pump Capacity (ton) = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("peak_combined_heatpump_thermal_ton"))
        else:
            for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])

NPV ($) =  -2820472.0
Capital Cost, Net ($) =  6534199.48
PV size_kw  =  0.0
Storage size_kw  =  0.0
Storage size_kwh  =  0.0
GHX Number of Boreholes =  1086.0
GHP Heat Pump Capacity (ton) =  463.345


## Do some custom response/results data processing

## Save API response and scenario data into a .pickle file for later use

In [27]:
response_save = api_response
file_name_to_save = "response_1"
with open("saved_responses/" + file_name_to_save + ".json", 'w') as fp:
    json.dump(response_save, fp)